# REM API Demo With `requests`

This notebook demonstrates how we can call the REM API to get an estimate
for the savings we can expect if we make an electrical upgrade to a home
at a particular address.

## Imports and Configuration

In [1]:
import requests
from pathlib import Path
import pandas as pd

In [2]:
HOST = "https://api.rewiringamerica.org"
REM_ADDRESS_URL = f"{HOST}/api/v1/rem/address"

API_KEY = None  # Put your API key here, or better yet in the file ~/.rwapi/api_key.txt

In [3]:
if API_KEY is None:
    api_key_path = Path.home() / ".rwapi" / "api_key.txt"

    if api_key_path.is_file():
        with open(api_key_path) as f:
            API_KEY = f.read().strip()

## Parameters

Address we are interested in and the upgrade we want to do.

In [4]:
address = "8009 Belmont Ave., Lubbock, TX 79424"
upgrade = "hvac__heat_pump_seer24_hspf13"
heating_fuel = "natural_gas"

## Make the Request

In [5]:
headers = {"Authorization": f"Bearer {API_KEY}"}

response = requests.get(
    url=REM_ADDRESS_URL,
    headers=headers,
    params=dict(address=address, upgrade=upgrade, heating_fuel=heating_fuel),
)

## Pull out the results

We are specifically interested in the total dollar savings.

In [6]:
data = response.json()

In [7]:
annual_savings = -data["fuel_results"]["total"]["delta"]["cost"]["mean"]["value"]

f"Expected annual savings: ${annual_savings:.2f}"

'Expected annual savings: $659.07'

## Modeling Only Baseline in REM

REM typically provides three estimates for a home: one without an upgrade, one with the requested upgrade, and the difference (delta) between them.
For a detailed explanation of these statistics, refer to the [All About REM Statistics notebook](https://github.com/rewiringamerica/api_demos/blob/main/notebooks/All%20About%20REM%20Statistics.ipynb).

A special case occurs when `baseline` is requested as the upgrade. In this case, REM returns a subset of the expected data structure.
For each `fuel_type` within `fuel_results`, the `upgrade` and `delta` fields will not be populated. 
Requesting `baseline` tells REM to perform estimation without applying any upgrades.

Performing a request with only `baseline` can be done similarly to other requests.

In [8]:
headers = {"Authorization": f"Bearer {API_KEY}"}

response = requests.get(
    url=REM_ADDRESS_URL,
    headers=headers,
    params=dict(address=address, upgrade="baseline", heating_fuel=heating_fuel),
)

### Pull out the results for a baseline request

We will extract the `fuel_results`, as we have done previously. In the response, `baseline` contains valid values, while `upgrade` and `delta` are null, as expected since no upgrade was applied to the home.

In [9]:
data = response.json()
fuel_results = data["fuel_results"]

In [10]:
# Convert the baseline from JSON to a pandas data frame for further
# analysis.
baseline = fuel_results[heating_fuel]["baseline"]


def stats_df(baseline):
    return pd.DataFrame(
        [
            {"metric": metric, "stat": stat} | value
            for metric, stats in baseline.items()
            for stat, value in stats.items()
        ]
    )


stats_df(baseline)

,metric,stat,value,unit
0,energy,mean,818.0488,therm
1,energy,median,773.5015,therm
2,energy,percentile_20,558.5560,therm
3,energy,percentile_80,1023.5165,therm
4,emissions,mean,5464.4159,kgCO2e
5,emissions,median,5166.8484,kgCO2e
6,emissions,percentile_20,3731.0518,kgCO2e
7,emissions,percentile_80,6836.9028,kgCO2e
8,cost,mean,1297.4921,$
9,cost,median,1241.7834,$


In [11]:
fuel_results[heating_fuel]["upgrade"]

In [12]:
fuel_results[heating_fuel]["delta"]

## Modeling Water Heater Upgrades

The `heating_fuel` of a home is always required by `/api/v1/rem/address`. When you request a water heater upgrade, you must also provide the fuel type of the existing water heater in the `water_heater_fuel` parameter.

In [13]:
# Model a heat pump water heater upgrade

response = requests.get(
    url=REM_ADDRESS_URL,
    headers=headers,
    params=dict(address=address, upgrade="water_heater__heat_pump_uef3.35", heating_fuel=heating_fuel, water_heater_fuel="electricity"),
)
data = response.json()
annual_savings = -data["fuel_results"]["total"]["delta"]["cost"]["mean"]["value"]

f"Expected annual savings: ${annual_savings:.2f}"

'Expected annual savings: $225.26'

You can optionally set `water_heater_fuel` for other upgrades as well. This can help make the results more accurate, by restricting the modeled homes to those with water heaters fueled in the same way.

However, if your water heater fuel type is relatively unusual for homes in your area with your heating fuel type, there is a chance the model won't be able to make a good prediction and you'll get an error message.

If this happens, we recommend not setting the water heater fuel type when it's not required.

In [14]:
# Attempt to model a home with an uncommon heating fuel and water heater fuel combination.

response = requests.get(
    url=REM_ADDRESS_URL,
    headers=headers,
    # It's very unusual to use propane for home heating with natural gas for water heating.
    params=dict(address=address, upgrade=upgrade, heating_fuel="propane", water_heater_fuel="natural_gas"),
)

print(response)

# The error message will let you know if the water heater fuel is likely to be the problem.
response.text

<Response [400]>


'{"detail":"The current version of this API cannot predict energy savings for the provided address. Omitting water_heater_fuel may help, by allowing more samples to match the characteristics of the provided home."}'

In [15]:
# Omit the water heater fuel to get results.
response = requests.get(
    url=REM_ADDRESS_URL,
    headers=headers,
    params=dict(address=address, upgrade=upgrade, heating_fuel="propane"),
)

response

<Response [200]>